<a href="https://colab.research.google.com/github/sharmarahulraj03/NLP_Tasks/blob/main/Assignment_4(Part_B)__DTI5125_Rahul_Raj_Sharma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1 align="center"><b>Data Science Applications</b></H1>
<H3 align="center"><b>Assignment 4</b></H3>
<H3 align="center"><b><u>Part : B</u></b></H3>
<H5 align="left"><b><u> Instruction:</u></b></H5>
<p align="justify">This assignment should be completed individually using either Python or R. Upon completion, present your result in 
one submission, including the report, answers generated and plots. Also, submit the source codes used to generate 
your results as a separate attachment.</p>
<H5 align="left"><b><u>Part B:   Semantic Analysis (50 Points)</u></b></H5>
<p align="justify">Named Entity Recognition (NER) enables the recognition of names of persons, locations, organizations, dates in text
documents. It involves two sub-tasks - identifying the boundaries of expressions (open & close brackets) and 
labeling the expressions with tags (e.g., PER, LOC or ORG). Using the reuters news dataset provided, complete the 
following:
<ol>
<li>Choose good features for encoding the problem and encode the dataset (10 points)</li>
<li> Run a classifier over the training dataset (10 points)</li>
<li> Fine tune model parameters (10 points)</li>
<li>
Perform evaluation, report the accuracy, precision, recall, F-measure (10 points)</li>
<li>
Return a confusion matrix (10 points)</li>
</ol></p>

**Importing required Libraries.**

In [49]:
import pandas as pd
import numpy as np
import spacy
import re
import matplotlib.pyplot as plt
import warnings
import os
import nltk 
import en_core_web_sm

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from spacy import displacy
from collections import Counter
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

warnings.filterwarnings('ignore')
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
#Reading the whole data
df = pd.read_excel("reuters_news_headlines.xlsx")
data_size = len(df)
print(data_size)
print(df.head())
print(df.tail())

32770
                                           Headlines  ...                                        Description
0  TikTok considers London and other locations fo...  ...  TikTok has been in discussions with the UK gov...
1  Disney cuts ad spending on Facebook amid growi...  ...  Walt Disney  has become the latest company to ...
2  Trail of missing Wirecard executive leads to B...  ...  Former Wirecard  chief operating officer Jan M...
3  Twitter says attackers downloaded data from up...  ...  Twitter Inc said on Saturday that hackers were...
4  U.S. Republicans seek liability protections as...  ...  A battle in the U.S. Congress over a new coron...

[5 rows x 3 columns]
                                               Headlines  ...                                        Description
32765  Malaysia says never hired British data firm at...  ...  The Malaysian government and the ruling party ...
32766  Prosecutors search Volkswagen headquarters in ...  ...  German prosecutors said on Tu

In [51]:
df

,Headlines,Time,Description
0,TikTok considers London and other locations fo...,Jul 18 2020,TikTok has been in discussions with the UK gov...
1,Disney cuts ad spending on Facebook amid growi...,Jul 18 2020,Walt Disney has become the latest company to ...
2,Trail of missing Wirecard executive leads to B...,Jul 18 2020,Former Wirecard chief operating officer Jan M...
3,Twitter says attackers downloaded data from up...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...
4,U.S. Republicans seek liability protections as...,Jul 17 2020,A battle in the U.S. Congress over a new coron...
...,...,...,...
32765,Malaysia says never hired British data firm at...,Mar 20 2018,The Malaysian government and the ruling party ...
32766,Prosecutors search Volkswagen headquarters in ...,Mar 20 2018,German prosecutors said on Tuesday they had se...
32767,McDonald's sets greenhouse gas reduction targets,Mar 20 2018,McDonald's Corp on Tuesday announced an approv...
32768,Pratt & Whitney to deliver spare A320neo engin...,Mar 20 2018,Pratt & Whitney will soon begin deliveries of ...


**Pre-Processing the Whole Data**

In [52]:
# Remove unwanted symbol
def remove_unwanted_characters(text):
    text = re.sub('\[[^]]*\]', " ", text)
    return text

df['Headlines'] = df['Headlines'].apply(remove_unwanted_characters)
df['Headlines'] 

0        TikTok considers London and other locations fo...
1        Disney cuts ad spending on Facebook amid growi...
2        Trail of missing Wirecard executive leads to B...
3        Twitter says attackers downloaded data from up...
4        U.S. Republicans seek liability protections as...
                               ...                        
32765    Malaysia says never hired British data firm at...
32766    Prosecutors search Volkswagen headquarters in ...
32767     McDonald's sets greenhouse gas reduction targets
32768    Pratt & Whitney to deliver spare A320neo engin...
32769    UK will always consider ways to improve data l...
Name: Headlines, Length: 32770, dtype: object

In [53]:
#change to lowercase
def remove_capitalization(text):
    lower_text= text.lower()
    return lower_text
df['Headlines'] = df['Headlines'].apply(remove_capitalization)
df['Headlines'] 

0        tiktok considers london and other locations fo...
1        disney cuts ad spending on facebook amid growi...
2        trail of missing wirecard executive leads to b...
3        twitter says attackers downloaded data from up...
4        u.s. republicans seek liability protections as...
                               ...                        
32765    malaysia says never hired british data firm at...
32766    prosecutors search volkswagen headquarters in ...
32767     mcdonald's sets greenhouse gas reduction targets
32768    pratt & whitney to deliver spare a320neo engin...
32769    uk will always consider ways to improve data l...
Name: Headlines, Length: 32770, dtype: object

In [54]:
def remove_numbers_punctuation(text):
    return re.sub('[^a-zA-Z,,.!:;]' , ' ', text)

def remove_brackets(text):
    return re.sub('\[[^]]*\]', "", text)

def remove_num_brac_punct(text):
    text = remove_numbers_punctuation(text)
    text = remove_brackets(text)
    return text

df['Headlines'] = df['Headlines'].apply(remove_num_brac_punct)
df['Headlines'] 

0        tiktok considers london and other locations fo...
1        disney cuts ad spending on facebook amid growi...
2        trail of missing wirecard executive leads to b...
3        twitter says attackers downloaded data from up...
4        u.s. republicans seek liability protections as...
                               ...                        
32765    malaysia says never hired british data firm at...
32766    prosecutors search volkswagen headquarters in ...
32767     mcdonald s sets greenhouse gas reduction targets
32768    pratt   whitney to deliver spare a   neo engin...
32769    uk will always consider ways to improve data l...
Name: Headlines, Length: 32770, dtype: object

In [55]:
def remove_white_space(text):
    return re.sub(r"\s\s+([^ \t\r\n])[ \t]+$", " ", text )
    #return re.sub(r"\s\s+",'  ', text)
    
df['Headlines'] = df['Headlines'].apply(remove_white_space)
df['Headlines'] 

0        tiktok considers london and other locations fo...
1        disney cuts ad spending on facebook amid growi...
2        trail of missing wirecard executive leads to b...
3        twitter says attackers downloaded data from up...
4        u.s. republicans seek liability protections as...
                               ...                        
32765    malaysia says never hired british data firm at...
32766    prosecutors search volkswagen headquarters in ...
32767     mcdonald s sets greenhouse gas reduction targets
32768    pratt   whitney to deliver spare a   neo engin...
32769    uk will always consider ways to improve data l...
Name: Headlines, Length: 32770, dtype: object

In [56]:
def tokenize_text(text):
    words = nltk.word_tokenize(text)
    return words

df['Headlines'] = df['Headlines'].apply(tokenize_text)
df['Headlines']  

0        [tiktok, considers, london, and, other, locati...
1        [disney, cuts, ad, spending, on, facebook, ami...
2        [trail, of, missing, wirecard, executive, lead...
3        [twitter, says, attackers, downloaded, data, f...
4        [u.s., republicans, seek, liability, protectio...
                               ...                        
32765    [malaysia, says, never, hired, british, data, ...
32766    [prosecutors, search, volkswagen, headquarters...
32767    [mcdonald, s, sets, greenhouse, gas, reduction...
32768    [pratt, whitney, to, deliver, spare, a, neo, e...
32769    [uk, will, always, consider, ways, to, improve...
Name: Headlines, Length: 32770, dtype: object

In [57]:
# Remove Stop Words
stopwords = nltk.corpus.stopwords.words('english')

def clean_stopwords(text):
    return [i for i in text if i not in stopwords]

df['Headlines'] = df['Headlines'].apply(clean_stopwords)
df['Headlines'] 

0        [tiktok, considers, london, locations, headqua...
1        [disney, cuts, ad, spending, facebook, amid, g...
2        [trail, missing, wirecard, executive, leads, b...
3        [twitter, says, attackers, downloaded, data, e...
4        [u.s., republicans, seek, liability, protectio...
                               ...                        
32765    [malaysia, says, never, hired, british, data, ...
32766    [prosecutors, search, volkswagen, headquarters...
32767    [mcdonald, sets, greenhouse, gas, reduction, t...
32768    [pratt, whitney, deliver, spare, neo, engines,...
32769    [uk, always, consider, ways, improve, data, la...
Name: Headlines, Length: 32770, dtype: object

In [58]:
lemma = WordNetLemmatizer()

def lemmatization(text):
    return [lemma.lemmatize(word=w, pos='v') for w in text]

df['Headlines'] = df['Headlines'].apply(lemmatization)
df['Headlines'] 

0        [tiktok, consider, london, locations, headquar...
1        [disney, cut, ad, spend, facebook, amid, grow,...
2        [trail, miss, wirecard, executive, lead, belar...
3        [twitter, say, attackers, download, data, eigh...
4        [u.s., republicans, seek, liability, protectio...
                               ...                        
32765    [malaysia, say, never, hire, british, data, fi...
32766    [prosecutors, search, volkswagen, headquarter,...
32767    [mcdonald, set, greenhouse, gas, reduction, ta...
32768    [pratt, whitney, deliver, spare, neo, engines,...
32769    [uk, always, consider, ways, improve, data, la...
Name: Headlines, Length: 32770, dtype: object

In [59]:
# convert tokens back to strings
def convert_token_to_string(text):
    return " ".join(text)

df['Headlines'] = df['Headlines'].apply(convert_token_to_string)
df['Headlines'] 

0             tiktok consider london locations headquarter
1        disney cut ad spend facebook amid grow boycott...
2        trail miss wirecard executive lead belarus , d...
3        twitter say attackers download data eight non ...
4        u.s. republicans seek liability protections co...
                               ...                        
32765    malaysia say never hire british data firm cent...
32766    prosecutors search volkswagen headquarter new ...
32767         mcdonald set greenhouse gas reduction target
32768    pratt whitney deliver spare neo engines soon i...
32769    uk always consider ways improve data laws : pm...
Name: Headlines, Length: 32770, dtype: object

In [60]:
df.head(5)

,Headlines,Time,Description
0,tiktok consider london locations headquarter,Jul 18 2020,TikTok has been in discussions with the UK gov...
1,disney cut ad spend facebook amid grow boycott...,Jul 18 2020,Walt Disney has become the latest company to ...
2,"trail miss wirecard executive lead belarus , d...",Jul 18 2020,Former Wirecard chief operating officer Jan M...
3,twitter say attackers download data eight non ...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...
4,u.s. republicans seek liability protections co...,Jul 17 2020,A battle in the U.S. Congress over a new coron...


In [61]:
df_clean = df

In [62]:
df_clean

,Headlines,Time,Description
0,tiktok consider london locations headquarter,Jul 18 2020,TikTok has been in discussions with the UK gov...
1,disney cut ad spend facebook amid grow boycott...,Jul 18 2020,Walt Disney has become the latest company to ...
2,"trail miss wirecard executive lead belarus , d...",Jul 18 2020,Former Wirecard chief operating officer Jan M...
3,twitter say attackers download data eight non ...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...
4,u.s. republicans seek liability protections co...,Jul 17 2020,A battle in the U.S. Congress over a new coron...
...,...,...,...
32765,malaysia say never hire british data firm cent...,Mar 20 2018,The Malaysian government and the ruling party ...
32766,prosecutors search volkswagen headquarter new ...,Mar 20 2018,German prosecutors said on Tuesday they had se...
32767,mcdonald set greenhouse gas reduction target,Mar 20 2018,McDonald's Corp on Tuesday announced an approv...
32768,pratt whitney deliver spare neo engines soon i...,Mar 20 2018,Pratt & Whitney will soon begin deliveries of ...


**1. Choose good features for encoding the problem and encode the dataset (10 points)**

In [63]:
nlp = en_core_web_sm.load()

In [64]:
df['Headlines']

0             tiktok consider london locations headquarter
1        disney cut ad spend facebook amid grow boycott...
2        trail miss wirecard executive lead belarus , d...
3        twitter say attackers download data eight non ...
4        u.s. republicans seek liability protections co...
                               ...                        
32765    malaysia say never hire british data firm cent...
32766    prosecutors search volkswagen headquarter new ...
32767         mcdonald set greenhouse gas reduction target
32768    pratt whitney deliver spare neo engines soon i...
32769    uk always consider ways improve data laws : pm...
Name: Headlines, Length: 32770, dtype: object

In [65]:
labels = []
for text in df["Headlines"]:
    en = nlp(text).ents
    labels.append([l.label_ for l in en])

In [66]:
labels

[['GPE'],
 ['ORG'],
 ['ORG', 'GPE', 'PERSON'],
 ['CARDINAL'],
 ['GPE', 'NORP'],
 ['TIME'],
 ['CARDINAL'],
 ['GPE', 'GPE'],
 ['ORG'],
 ['GPE'],
 ['ORG'],
 [],
 ['ORG'],
 ['PERSON'],
 [],
 ['ORG', 'DATE', 'ORG'],
 ['ORG', 'ORG'],
 ['ORG'],
 ['ORG', 'GPE'],
 [],
 [],
 ['ORG'],
 ['PERSON', 'GPE'],
 ['GPE'],
 ['GPE'],
 ['ORG', 'ORG', 'PERSON'],
 ['PERSON', 'GPE'],
 ['GPE', 'NORP'],
 ['ORG'],
 [],
 ['GPE'],
 ['ORG', 'ORG'],
 [],
 [],
 [],
 [],
 ['ORG'],
 ['ORG'],
 [],
 [],
 ['GPE'],
 ['NORP'],
 ['ORG', 'GPE'],
 ['ORG', 'DATE'],
 [],
 ['NORP', 'PERSON', 'CARDINAL'],
 ['ORG', 'PERSON'],
 [],
 ['ORG'],
 ['ORG', 'GPE', 'DATE'],
 ['ORG', 'GPE'],
 ['ORG'],
 ['ORG'],
 ['ORG', 'PERSON'],
 [],
 [],
 [],
 ['ORG', 'DATE'],
 ['GPE', 'CARDINAL'],
 ['GPE'],
 [],
 ['GPE', 'GPE'],
 [],
 ['ORG'],
 ['ORG'],
 ['GPE'],
 ['GPE', 'ORG'],
 [],
 ['ORG'],
 ['ORG', 'NORP'],
 ['ORG'],
 ['ORG'],
 ['GPE'],
 ['ORG', 'GPE'],
 [],
 ['GPE'],
 [],
 ['GPE', 'ORG'],
 ['ORG'],
 ['GPE'],
 ['GPE', 'ORG'],
 [],
 ['PERSON', 'GPE', 

In [67]:
label_bool = []
for idx, lab in enumerate(labels):
    if len(lab)!=0:
        label_bool.append(True)
    else:
        label_bool.append(False)

In [68]:
df['labels'] = labels

In [69]:
df = df.loc[label_bool,:]

In [70]:
mlb = MultiLabelBinarizer()

res = pd.DataFrame(mlb.fit_transform(df['labels']),
                   columns=mlb.classes_,
                   index=df.index)

In [71]:
res.head(10)

,CARDINAL,DATE,EVENT,FAC,GPE,LANGUAGE,LAW,LOC,MONEY,NORP,ORDINAL,ORG,PERCENT,PERSON,PRODUCT,QUANTITY,TIME,WORK_OF_ART
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
res.sum()

CARDINAL        2610
DATE            3104
EVENT             29
FAC               70
GPE            13210
LANGUAGE          16
LAW                8
LOC              563
MONEY             78
NORP            2966
ORDINAL          442
ORG            14622
PERCENT           18
PERSON          4386
PRODUCT          322
QUANTITY          27
TIME              80
WORK_OF_ART        5
dtype: int64

In [73]:
res = res[['CARDINAL','DATE', 'GPE','NORP','ORG','PERSON']]

In [74]:
res.head()

,CARDINAL,DATE,GPE,NORP,ORG,PERSON
0,0,0,1,0,0,0
1,0,0,0,0,1,0
2,0,0,1,0,1,1
3,1,0,0,0,0,0
4,0,0,1,1,0,0


In [75]:
df

,Headlines,Time,Description,labels
0,tiktok consider london locations headquarter,Jul 18 2020,TikTok has been in discussions with the UK gov...,[GPE]
1,disney cut ad spend facebook amid grow boycott...,Jul 18 2020,Walt Disney has become the latest company to ...,[ORG]
2,"trail miss wirecard executive lead belarus , d...",Jul 18 2020,Former Wirecard chief operating officer Jan M...,"[ORG, GPE, PERSON]"
3,twitter say attackers download data eight non ...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...,[CARDINAL]
4,u.s. republicans seek liability protections co...,Jul 17 2020,A battle in the U.S. Congress over a new coron...,"[GPE, NORP]"
...,...,...,...,...
32765,malaysia say never hire british data firm cent...,Mar 20 2018,The Malaysian government and the ruling party ...,"[GPE, NORP]"
32766,prosecutors search volkswagen headquarter new ...,Mar 20 2018,German prosecutors said on Tuesday they had se...,[ORG]
32767,mcdonald set greenhouse gas reduction target,Mar 20 2018,McDonald's Corp on Tuesday announced an approv...,[ORG]
32768,pratt whitney deliver spare neo engines soon i...,Mar 20 2018,Pratt & Whitney will soon begin deliveries of ...,[GPE]


In [76]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,max_features=150,norm='l2', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df['Headlines']).toarray()

In [77]:
features_df = pd.DataFrame(features)
features_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.603089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.501024,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.84314,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.537694,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.639042,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.769172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**2. Fine tune model parameters (10 points)**

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_df, res, test_size=0.15, random_state=42)

In [79]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 200, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 51, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [80]:
random_grid

{'bootstrap': [True, False],
 'max_depth': [10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 51, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100,
  105,
  110,
  115,
  121,
  126,
  131,
  136,
  142,
  147,
  152,
  157,
  163,
  168,
  173,
  178,
  184,
  189,
  194,
  200]}

In [81]:
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(features, res)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [82]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 22,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 173}

**3. Run a classifier over the training dataset (10 points)** 

In [83]:
rf = RandomForestClassifier(n_estimators= 173,min_samples_split= 5,min_samples_leaf= 4,max_features= 'sqrt',max_depth= 22,bootstrap= False)


In [84]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=22, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=173,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [85]:
predicted = rf.predict(X_test)
y_true = y_test.to_numpy()

In [86]:
class_names = ['CARDINAL','DATE', 'GPE','NORP','ORG','PERSON']

**4. Perform evaluation, report the accuracy, precision, recall, F-measure (10 points)**

In [87]:
f1score = f1_score(y_true, predicted, zero_division=1,average='samples')
print('The F1 Score is',f1score)
acc = accuracy_score(y_true, predicted)
print('The Accuracy Score is',acc)
recall = recall_score(y_true, predicted, average='samples')
print('The Recall Score is',recall)
precision = precision_score(y_true, predicted, average='samples')
print('The Precision Score is',precision)

The F1 Score is 0.5818629434776743
The Accuracy Score is 0.3890462700661001
The Recall Score is 0.5442437834435001
The Precision Score is 0.6698929807994964


**5. Return a confusion matrix (10 points)**

In [88]:
matrix = confusion_matrix(y_true.argmax(axis=1), predicted.argmax(axis=1))

In [89]:
print(matrix)

[[ 185   17   45    3  181    0]
 [  25  122   87    5  209    0]
 [ 112   10  819   18  708    0]
 [  26    3   27   56  160    0]
 [  63    6   56    4 1058    0]
 [  22    3   10    0  196    0]]


<H5 align="left"><b> Thank you </b></H5>
<H5 align="left"><b> Assignment -4 Part B: Semantic Analysis </b></H5>
<H5 align="left"><b> Prepared and Submitted By : Rahul Raj Sharma </b></H5>